Testing SuperK SELECT control via Python. We are using NKT Photonics SuperK Compact laser with SELECT tunable filter. Laser is on COM3.

On Rayleigh, SDK documents can be found under Users/Public/Public Documents/NKT Photonics

SuperK Compact info is on page 59 of the user manual.

Python interface doesn't work while Control software is connected.

Notes for Compact registers: 

- The module type number is 74h. The standard module address is 1 (01h).
- Emission on/off – the value 0 turns emission off, and 1 turns it on (if interlock circuit
has been reset). The format is an 8-bit unsigned integer 
- Trig mode set with 31h
- Register 0x30 is emission. Register 0x99 might be power? Value 100 for 100%
- Interlock register is 32h. A value greater than 1 will reset the interlock circuit if the key is in the "On" position. Sending 0 will disable the interlock. 
- Internal pulse frequency (repetition rate) is 33h. The value is in Hz. Minimum value is 1, maximum is system dependent
- Internal pulse frequency limit readout with 36h. 
- Power level 3Eh. Setpoint value in %. Values input here are automatically converted to repetition rates which overwrites the value in 33h register. Integer value between 0 and 100
- Status bits at 66h
- Readout display text with 78h. 


- Should also look into specific registers for SuperK SELECT


In [7]:
import serial
import time

# Configure the serial connection
com_port = 'COM3'  # Change if necessary
baud_rate = 9600    # Set the appropriate baud rate for your device

# Initialize the serial connection
try:
    laser = serial.Serial(com_port, baud_rate, timeout=1)
    print(f"Connected to {com_port}")

    # Give the connection a moment to settle
    time.sleep(2)

    # Command to turn on the laser (this command may vary; check the manual)
    turn_on_command = 'LASE:ON\r\n'  # Example command; replace with the correct one

    # Send the command
    laser.write(turn_on_command.encode('utf-8'))
    print("Laser turned on.")

except serial.SerialException as e:
    print(f"Error: {e}")

finally:
    # Close the connection when done
    if 'laser' in locals() and laser.is_open:
        laser.close()
        print("Connection closed.")

Connected to COM3
Laser turned on.
Connection closed.


In [3]:
import pylablib as pll

In [8]:
pll.list_backend_resources("serial")

['COM3', 'COM1']

In [10]:
from pylablib import devices

In [1]:
import pylablib.devices as devices
print(dir(devices))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [2]:
from pylablib.devices import NKT

ImportError: cannot import name 'NKT' from 'pylablib.devices' (C:\Users\EPL User\AppData\Roaming\Python\Python310\site-packages\pylablib\devices\__init__.py)

In [4]:
from NKTP_DLL import *

Loading x64 DLL from: C:\Users\Public\Documents\NKT Photonics\SDK\NKTPDLL\x64\NKTPDLL.dll


In [11]:
# For Extreme
result = registerWriteU8('COM27', 15, 0x30, 3, -1)
print('Setting emission ON - Extreme:', RegisterResultTypes(result))

Setting emission ON - Extreme: 13:RegResultPortNotFound


I think the interlock key has to be turned and reset first. Find a way to check interlock status?

In [71]:
# try my version with Compact
# format goes (port, module address 15=0x0F, emission control register=0x30, data value, ???)
result = registerWriteU8('COM3', 1, 0x30, 1, -1)
print('Setting emission ON - Compact:', RegisterResultTypes(result))

Setting emission ON - Compact: 0:RegResultSuccess


In [5]:
def power_on():
    result = registerWriteU8('COM3', 1, 0x30, 1, -1)
    print('Setting emission ON.', RegisterResultTypes(result))

In [6]:
def power_off():
    result = registerWriteU8('COM3', 1, 0x30, 0, -1)
    print('Setting emission OFF.', RegisterResultTypes(result))

In [9]:
power_off()

Setting emission OFF. 0:RegResultSuccess


In [53]:
# Turn laser off
result = registerWriteU8('COM3', 1, 0x30, 0, -1)
print('Setting emission OFF - Compact:', RegisterResultTypes(result))

Setting emission OFF - Compact: 0:RegResultSuccess


In [70]:
# Set laser power
result = registerWriteU8('COM3', 1, 0x99, 50, -1)
print('Setting power - Compact:', RegisterResultTypes(result))

Setting power - Compact: 4:RegResultNacked


In [48]:
# Check power level
result = registerReadU8('COM3', 1, 0x3E, 0) # last value is the channel? 1 through 8. Second value in the output is power level for that channel maybe and first is overall?
# 0 value gives output (0, 20). The power level is in fact 20%! checked
print(result)
print(RegisterResultTypes(result))

(0, 25)
Unknown result


In [ ]:
# Check power level, works
def get_overall_power():
    result = registerReadU8('COM3', 1, 0x3E, 0) # devID=1 for Compact
    power = result[1]
    print(f'Overall power level: {power}%')

In [ ]:
# Set power level, works
def set_overall_power(power):
    result = registerWriteU8('COM3', 1, 0x3E, power, -1)
    print(f'Setting overall power level to {power}%: {RegisterResultTypes(result)}')

In [75]:
set_overall_power(50)

Setting overall power level to 50%: 0:RegResultSuccess


In [76]:
get_overall_power()

Overall power level: 50%


In [66]:
# Readout of the maximum permissible internal frequency in Hz
result = registerReadF32('COM3', 1, 0x36, -1)
print(result)
print(RegisterResultTypes(result))

(0, 3.08467830951822e-41)
Unknown result


In [65]:
# Readout interlock
result = registerReadF32('COM3', 1, 0x32, -1)
print(result)
print(RegisterResultTypes(result))

(8, 2.802596928649634e-45)
Unknown result


In [ ]:
def interlock_status():
    if value==0:
        print('Interlock off')
    if value

In [ ]:
status_mapping = {
    0: "Device is off.",
    1: "Device is on.",
    2: "Device is broken."
}

# Function to check the status based on the mapping
def check_device_status(value):
    # Use the get method to provide a default message for unknown values
    message = status_mapping.get(value, "Unknown status.")
    print(message)

# Example usage
status_value = int(input("Enter the device status (0: off, 1: on, 2: broken): "))
check_device_status(status_value)

In [ ]:
interlock_mapping = {
    0: "Interlock off (interlock circuit open).",
    1: "Waiting for interlock reset.",
    3: "Interlock is OK."
}

def interlock_status(value):
    message = interlock_mapping.get(value, "Unknown interlock status.")
    print(message)

 0 Interlock off (interlock circuit open)
0 1 Waiting for interlock reset
0 2 Interlock is OK
1 0 Front panel interlock / key switch off
2 0 Door switch open
3 0 External module interlock
4 0 Interlock power failure
255 - Interlock circuit failure